In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import useful_functions as uf
import pandas as pd
import coin_circle_detector as ccd

In [2]:
folder = 'data'
images = [f for f in os.listdir(folder) if f.endswith('.jpg')]

In [3]:
design = uf.generate_design({'dp':[.5, 1.5, 11, 1], 'param2':[5, 15, 11,0]}, rename_index=True)
print(f'Número de modelos: {len(design)}')

Número de modelos: 121


In [4]:
from statistics import mean

def encontrar_sublista(lista, posicion):
    acumulador = 0
    for i, sublista in enumerate(lista):
        if acumulador <= posicion < acumulador + len(sublista):
            return i  # Retorna el índice de la sublista donde está la posición
        acumulador += len(sublista)
    return None  # Si la posición no está en la lista

def z_score_abs(array):
    media = np.mean(array)
    desviacion = np.std(array, ddof=0)  # Desviación estándar de la población
    return abs((array - media) / desviacion)


def coeficiente_variacion(array):
    return array.std(ddof=1) / array.mean()

bests = ['best_1', 'best_2', 'best_3']
worsts = ['worst_1', 'worst_2', 'worst_3']


for index , row in design.iterrows():
    dp_, param2_ = row[0], row[1]
    radios_monedas = {}
    monedas = ['tr_005', 'tr_01', 'tr_02', 'tr_05', 'tr_1', 'tr_2']
    n_monedas = 0
    
    best_3 = {}
    worst_3 = {}

    resultados = {}
    for moneda_index in monedas:
        radios = []
        img_files_filter = list(filter(lambda x: moneda_index in x, images)) # None tr_005 tr_01 tr_02 tr_05 tr_1 tr_2
        uf.sort_list_of_fs_by_ascending_number(img_files_filter, r_pattern='image_')
        for img_n in img_files_filter:
            _,_,circ = ccd.coins_detec_alg(f'{folder}/{img_n}', vis_result=False, vis_proc = False, params={'dp':dp_, 'param2':param2_})
            if circ is not None:
                radios.append(list(circ[:,2]))
        radios_len = len(sum(radios, []))
        if radios_len > 30:
            radios_np_array = np.array(sum(radios, []))

            n_monedas += radios_len
            
            cv = coeficiente_variacion(radios_np_array)
            resultados[moneda_index] = cv
            
            worst_val = z_score_abs(radios_np_array).max() 
            ind_max = z_score_abs(radios_np_array).argmax()
            worst_img = img_files_filter[encontrar_sublista(radios, ind_max)]
            worst_3[worst_img] = worst_val
            
            best_val = z_score_abs(radios_np_array).min() 
            ind_min = z_score_abs(radios_np_array).argmin()
            best_img = img_files_filter[encontrar_sublista(radios, ind_min)]
            best_3[best_img] = best_val
            
        else:
            cv = None
    
    worst_3 = sorted(worst_3.items(), key=lambda x: x[1], reverse=True)[:3]
    best_3 = sorted(best_3.items(), key=lambda x: x[1])[:3]
    cv_mean = mean(resultados.values()) if resultados else None
    design.loc[index, 'cv_mean'] = cv_mean
    design.loc[index, 'n_coins'] = int(n_monedas)
       
    if best_3:
        for n, best_n in enumerate(best_3):
            design.loc[index, bests[n]] = best_n[0].split('.')[0]
            design.loc[index, f'{bests[n]}_val'] = best_n[1]
            
    if worst_3:
        for n, worst_n in enumerate(worst_3):
            design.loc[index, worsts[n]] = worst_n[0].split('.')[0]
            design.loc[index, f'{worsts[n]}_val'] = worst_n[1]

design


,dp,param2,cv_mean,n_coins,best_1,best_1_val,best_2,best_2_val,best_3,best_3_val,worst_1,worst_1_val,worst_2,worst_2_val,worst_3,worst_3_val
Model_1,0.5,5.0,0.126568,516.0,tr_01_image_1,0.006028,tr_1_image_1,0.022862,tr_005_image_1,0.047212,tr_01_image_15,5.160248,tr_1_image_11,5.147797,tr_02_image_8,4.927603
Model_2,0.5,6.0,0.065979,467.0,tr_01_image_1,0.007372,tr_005_image_1,0.007388,tr_1_image_1,0.095231,tr_01_image_2,7.511861,tr_02_image_6,7.375267,tr_005_image_3,6.479077
Model_3,0.5,7.0,0.029434,453.0,tr_1_image_1,0.075550,tr_005_image_1,0.093911,tr_2_image_1,0.142196,tr_1_image_11,8.647051,tr_005_image_5,4.539014,tr_2_image_13,2.749120
Model_4,0.5,8.0,0.026115,452.0,tr_005_image_1,0.093911,tr_2_image_1,0.142196,tr_05_image_1,0.313314,tr_005_image_5,4.539014,tr_1_image_16,3.072407,tr_2_image_13,2.749120
Model_5,0.5,9.0,0.026131,452.0,tr_005_image_1,0.093911,tr_2_image_1,0.142196,tr_05_image_1,0.313314,tr_005_image_5,4.539014,tr_1_image_16,3.072407,tr_2_image_13,2.749120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Model_117,1.5,11.0,0.118019,495.0,tr_02_image_1,0.013679,tr_005_image_1,0.037636,tr_2_image_2,0.049311,tr_1_image_15,6.908710,tr_02_image_7,6.251287,tr_005_image_10,5.696525
Model_118,1.5,12.0,0.080735,471.0,tr_02_image_1,0.013679,tr_05_image_1,0.014290,tr_005_image_1,0.069900,tr_005_image_10,6.623056,tr_02_image_7,6.251287,tr_05_image_7,6.216189
Model_119,1.5,13.0,0.059737,458.0,tr_02_image_1,0.085089,tr_01_image_1,0.103227,tr_05_image_1,0.190368,tr_02_image_4,6.381650,tr_1_image_10,5.918157,tr_005_image_3,5.630156
Model_120,1.5,14.0,0.053348,456.0,tr_02_image_1,0.085089,tr_01_image_1,0.113828,tr_05_image_1,0.190368,tr_02_image_4,6.381650,tr_01_image_4,5.261399,tr_1_image_12,4.875251


In [5]:
print(design['cv_mean'].min())
print(design['cv_mean'].idxmin())

dp_opt = design.loc[design['cv_mean'].idxmin(), 'dp']
param2_opt = design.loc[design['cv_mean'].idxmin(), 'param2']

print(f'Parametros: dp: {dp_opt} | dp: {param2_opt} ')



0.02611547682745448
Model_4
Parametros: dp: 0.5 | dp: 8.0 


In [6]:
design.to_excel('design_results.xlsx') # =INDICE($A:$A;COINCIDIR(MIN($D:$D);$D:$D;0))

/var/folders/kb/2kjmxrfs0vn8rt7y7z6l4mwm0000gn/T/ipykernel_81112/1745687354.py:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.9' currently installed).
  design.to_excel('design_results.xlsx') # =INDICE($A:$A;COINCIDIR(MIN($D:$D);$D:$D;0))


In [7]:
# img_n = 'tr_01_image_2.jpg'
# _,_,circ = ccd.alg_2(f'{folder}/{img_n}', vis_result=True, vis_proc = False, params={'dp':.5, 'param2':11})